In [ ]:
import pandas as pd
from bokeh.layouts import row
from bokeh.plotting import figure, output_notebook, show
from sklearn.ensemble import RandomForestRegressor
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
import numpy as np;

In [ ]:
output_notebook()

In [ ]:
file_name_model_data = "../model_data/SimulatedChillerPerformanceData.gzip"

In [ ]:
newTable=pd.read_csv(file_name_model_data, compression='gzip' ,encoding="ISO-8859-2")

In [ ]:
newTable[['Db','Avlblty']].describe()

In [ ]:
dataTable=newTable[['Db','Rh', 'Baro', 'Avlblty', 'Cit', 'Ton', 'Load', 'EvapIn','EvapOut']]

In [ ]:
def auxmw(x,ton,threshold,adder):
    if x == 0:
        aux=0        
        return aux
    elif x<= ton:
        aux=0.0002*x**2 + 0.0922*x + 252.66 +500*adder
    
    else:
        aux=((0.0002*(threshold)**2 + 0.0922*(threshold) + 252.66)+(0.0002*(x-threshold)**2 + 
                                                                  0.0922*(x-threshold) + 252.66))+750*adder
    return aux

In [ ]:
# Add 1 to 1.5 MW  aux load
dataTable['new_Load'] = (dataTable['Ton'].apply(lambda x: auxmw(x,3300,3000,0.5)))/1000

In [ ]:
# aux=dataTable['new_Load']
# ton=dataTable['Ton']
# output_notebook()
# t1 = figure(plot_width=350, plot_height=350,title="aux Plot", x_axis_label='Index', 
#             y_axis_label='aux')
# t1.scatter(ton, aux)
# show(t1)

In [ ]:
X=dataTable[['Db', 'Rh', 'Baro', 'Avlblty']].values
y_cit=dataTable['Cit'].values.reshape(-1,1)
y_aux=dataTable['new_Load'].values.reshape(-1,1)
y_evap=dataTable['EvapOut'].values.reshape(-1,1)
y_ton=dataTable['Ton'].values.reshape(-1,1)

In [ ]:
dataTable[['new_Load','Rh','Avlblty']].describe()

In [ ]:
def model(layers=(40,40),es=True,n_iter=200,tol=0.0001):
    ppl=Pipeline([('scaler',StandardScaler()),
                  ('estimator',MLPRegressor(hidden_layer_sizes=layers,
                                            early_stopping=es,tol=tol,max_iter=n_iter,random_state=2301))
                 ])
    return ppl

In [ ]:
cit_= model(layers=(80,80,80))
cit_=cit_.fit(X,y_cit)

In [ ]:
aux_= model(layers=(80,80,80))
aux_=aux_.fit(X,y_aux)

In [ ]:
ton_= model(layers=(100,100,100),es=False,n_iter=50,tol=10)
ton_=ton_.fit(X,y_ton)

In [ ]:
evap_= model(es=True)
evap_=evap_.fit(X,y_evap)

In [ ]:
# cit_pred=cit_.predict(X)
# aux_pred=aux_.predict(X)
# ton_pred=ton_.predict(X)
# '''
# creating a new plot with a title and axis labels. 
# '''
# p1 = figure(plot_width=350, plot_height=350,title="actual vs predicted CIT", x_axis_label='actual', 
#             y_axis_label='predicted')
# p2 = figure(plot_width=350, plot_height=350,title="actual vs predicted aux", x_axis_label='actual', 
#             y_axis_label='predicted')
# p3 = figure(plot_width=350, plot_height=350,title="actual vs predicted Ton", x_axis_label='actual', 
#             y_axis_label='predicted')

# p1.scatter(y_cit.reshape(-1),cit_pred),p2.scatter(y_aux.reshape(-1),aux_pred)
# p3.scatter(y_ton.reshape(-1),ton_pred)

In [ ]:
# show(row(p1,p2,p3))

In [ ]:
pkl = "../../../pickles/chiller.pkl"
models = {
            'chillercit<cit><db|rh|baro|avlblty>'        :cit_,
            'chillerload<load><db|rh|baro|avlblty>'      :aux_,
            'chillerton<ton><db|rh|baro|avlblty>'        :ton_,
            'chillerevap<evap_out><db|rh|baro|avlblty>'  :evap_
         }
with open(pkl, "wb") as f:
    joblib.dump(models, f)
print(f'{f.name}')


In [ ]:
cit_.predict(np.array([95,0.5,14.7,100]).reshape(1,-1))

In [ ]:
ton_.predict(np.array([95,0.5,14.7,6500]).reshape(1,-1))